In [3]:
import os
import re

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
DATASET_DIR = "datasets/utkface/"
IMG_DIM = 200

In [ ]:
# Find files missing ethnicity label (implemented in utils.prepare_dataset())

pattern = re.compile(r'^(\d+)_(\d)_(\d)_\d{16}')
bad_files = []

for file in os.listdir(DATASET_DIR):
    if file.endswith(".jpg"): 
        if not pattern.match(file):
            bad_files.append(file)
            print(f"File does not match expected format: {file}")

if not bad_files:
    print("All files match the expected format.")

File does not match expected format: 39_1_20170116174525125.jpg.chip.jpg
File does not match expected format: 61_1_20170109142408075.jpg.chip.jpg
File does not match expected format: 61_1_20170109150557335.jpg.chip.jpg
